# Asynchronous worker

The `query.py` submodule has an asynchronous worker function `ibmpairs.query.batch_query()` that allows for the execution of an array of query objects in a sliding window of an arbitrary number at a time.

`ibmpairs.query.batch_query()` uses asyncio to create a queue of items from a provided array and executes them asynchronously through the completion, when one query completes another is pulled from the array. 

Given the requirement to run many queries of the raster and 'batch' or 'converted' point types, the use of the asynchronous worker function can speed up the process of `submit()`, `status()` and `download()` as the majority of the time spent in wait is in the checking status.

By default the asynchronous worker can be set to use up to 8 workers to act upon a list of queries in a sliding window fashion.

In [ ]:
%pip install ibmpairs
%pip install configparser

In [1]:
import os
import ibmpairs.query as query
import ibmpairs.client as client

import configparser

config = configparser.RawConfigParser()
config.read('../../../auth/secrets.ini')
# Best practice is not to include secrets in source code so we read
# an api key, tenant id and org id from a secrets.ini file.
# You could set the credentials in-line here but we don't
# recommend it for security reasons.

EI_API_KEY    = config.get('EI', 'api.api_key')
EI_TENANT_ID  = config.get('EI', 'api.tenant_id') 
EI_ORG_ID     = config.get('EI', 'api.org_id') 

ei_client = client.get_client(api_key   = EI_API_KEY,
                              tenant_id = EI_TENANT_ID,
                              org_id    = EI_ORG_ID)

raster_dict = {
  "layers": [
    {
      "id": "49429",
      "type": "raster"
    }
  ],
  "spatial": {
    "coordinates": [
      "35.6523",
      "-87.62",
      "51.6523",
      "-71.62"
    ],
    "type": "square"
  },
  "temporal": {
    "intervals": [
      {
        "start": "2023-01-01T00:00:00Z",
        "end":   "2023-01-01T00:00:00Z"
      }
    ]
  }
}

from datetime import datetime
start = datetime(2023,1,2,0,0,0)
end   = datetime(2023,1,2,0,0,0)

raster_query_batch          = query.query_from_dict(raster_dict)
raster_query_batch_next_day = query.query_from_dict(raster_dict)
raster_query_batch_next_day.replace_dates(start_date = start,
                                          end_date   = end)
queries = [raster_query_batch, raster_query_batch_next_day]

result = query.batch_query(queries = queries,
                           workers = 2,
                           verify = False)

2024-07-31 17:13:19 - paw - INFO - The client authentication method is assumed to be OAuth2.
2024-07-31 17:13:19 - paw - INFO - Legacy Environment is False
2024-07-31 17:13:19 - paw - INFO - The authentication api key type is assumed to be IBM Cloud IAM, because the api key prefix 'PHX' is not present.
2024-07-31 17:13:20 - paw - INFO - Authentication success.
2024-07-31 17:13:20 - paw - INFO - HOST: https://api.ibm.com/geospatial/run/na/core/v3
2024-07-31 17:13:20 - paw - INFO - TASK: batch_query STARTING.


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10844a550>


2024-07-31 17:13:20 - paw - INFO - The status must be checked in order to query and download.
2024-07-31 17:13:20 - paw - INFO - The status must be checked in order to query and download.
2024-07-31 17:13:21 - paw - INFO - The query was successfully submitted with the id: 1722427200_15201227.
2024-07-31 17:13:21 - paw - INFO - The query was successfully submitted with the id: 1722427200_15201199.
2024-07-31 17:13:21 - paw - INFO - The query 1722427200_15201199 has the status Queued.
2024-07-31 17:13:21 - paw - INFO - The query 1722427200_15201227 has the status Queued.
2024-07-31 17:13:52 - paw - INFO - The query 1722427200_15201199 has the status Succeeded.
2024-07-31 17:13:52 - paw - INFO - The query 1722427200_15201199 was successful after checking the status.
2024-07-31 17:13:53 - paw - INFO - The query 1722427200_15201227 has the status Succeeded.
2024-07-31 17:13:53 - paw - INFO - The query 1722427200_15201227 was successful after checking the status.
2024-07-31 17:14:22 - paw - 

In [2]:
print(result)

[{
    "download_file_name": "1722427200_15201227",
    "download_folder": "/ibmpairs/download/",
    "download_status": "SUCCEEDED",
    "id": "1722427200_15201227",
    "layers": [
        {
            "id": "49429",
            "type": "raster"
        }
    ],
    "merge_response": {},
    "spatial": {
        "coordinates": [
            35.6523,
            -87.62,
            51.6523,
            -71.62
        ],
        "type": "square"
    },
    "status_response": {
        "flag": false,
        "id": "1722427200_15201227",
        "ne_lat": 51.6523,
        "ne_lon": -71.62,
        "ready": true,
        "rt_status": "Succeeded",
        "start": 1722442401227,
        "status": "Succeeded",
        "status_code": 20,
        "sw_lat": 35.6523,
        "sw_lon": -87.62
    },
    "submit_response": {
        "id": "1722427200_15201227"
    },
    "temporal": {
        "intervals": [
            {
                "end": "2023-01-01T00:00:00Z",
                "start": "20

The async feature can also be used within a Jupyter Notebook where it hooks into the asyncio loop that operates the notebook.